## Random Forest

Summary: In this model, we tuned two parameters number of trees and max deepth for each tree based on training data. We selected model based on the score for each model. According to 5 folds cross validation, the best model has 400 trees and max depth as 13. The score for this model is 0.68. Then, we apply this model to test data. The accuracy for prediction is 0.6862.

Random forests or random decision forests are an ensemble learning method for classification and regression that operate by constructing a multitude of decision trees at training time and outputting the class that is the mode of the classes (classification) or mean prediction (regression) of the individual trees. Random decision forests gives a  robust result and avoid overfitting.

In [1]:
import pandas as pd

In [2]:
Xtrain=pd.read_csv('X_train.csv')

In [12]:
ytrain=pd.read_csv('y_train.csv',header=None)

In [3]:
import sklearn.ensemble as kn

In [7]:
rf=kn.RandomForestClassifier()

In [4]:
import numpy as np

In [5]:
from sklearn.model_selection import GridSearchCV

For random forest, we should tune two parameters: number of trees and max deepth for each tree.  We use five folders cross validation in trainning set to choose the best parameters combination. We choose the best combination by best_socre. The higher the score, the better the model. Thus, we choose the model with the highest score.

In [7]:
tree_num=[50,100,150,200,250,300,350,400,450,500,550,600] #n_estimators

In [8]:
max_depth=[7,8,9,10,11,12,13] #max_depth

In [9]:
parameters ={'n_estimators':tree_num,'max_depth':max_depth}

In [10]:
clf = GridSearchCV(rf,parameters,cv=5) #cv

In [22]:
clf.fit(X_train,y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'n_estimators': [50, 100, 150, 200, 250, 300, 350, 400], 'max_depth': [7, 8, 9, 10, 11, 12, 13]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [23]:
clf.best_estimator_

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [25]:
clf.best_params_

{'max_depth': 13, 'n_estimators': 400}

In [26]:
clf.best_score_

0.68001083790557471

From the result, we can find the best parameters are max_depth=13 and n_estimators=400. The best score is about 0.68.

Then, we predict label for test set.

In [8]:
best_model=kn.RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [15]:
best_model.fit(train,np.array(ytrain))

C:\Program Files\Anaconda2\lib\site-packages\ipykernel\__main__.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  if __name__ == '__main__':


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=13, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=400, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [17]:
y_hat=best_model.predict(X_test) ##label 

In [18]:
y_prob=best_model.predict_proba(X_test) #prob

In [23]:
acc=np.mean([y_hat[i]==y_test.iloc[i,0] for i in range(len(y_hat))]) #accuracy

The accuracy of the model is 68.62%.

## Model Comparasion

The predictions of these three models are similiar. The accuracy for svm, random forest and sgboost are 67%, 68% and 70%. For all of these three models, the predominants are 7(NDF) and 10(USA). Especially, svm and random forest predict the labels are as 7 and 10 which xgboost have several prediction as other countries like 3(ES) and 4(FR). Thus, we can see that the model predict 7 and 10 well but bad for other label. One reason is because of the unbalance in the proportion of the labels. Since the predominant of NDF and USA, the model will sacrifice other labels to ensure the high performance in predicting these two labels.

## Model Ensembling

In statistics and machine learning, ensemble methods use multiple learning algorithms to obtain better predictive performance than could be obtained from any of the constituent learning algorithms alone.Unlike a statistical ensemble in statistical mechanics, which is usually infinite, a machine learning ensemble refers only to a concrete finite set of alternative models, but typically allows for much more flexible structure to exist among those alternatives.

We use majority voting to determine final result. We set weight according to accuracy. For each model i with accuracy $a_i$, the weight $w_i=round(\frac{a_i}{\sum a_i}*100)$. 

Accoding to out three accuracy, we set the weight as 33,34,33 for randomforest, svm and xgboost.

In [32]:
import collections as cls

In [33]:
def pred_ensembel(preds,weights):
    '''This function will get the final predict by ensembling
    Input:
        preds: prediction of models
        weights:weight for each model
    '''
    weighted_pred=[]
    n=preds.shape[0]
    p=preds.shape[1]
    for i in range(n):
        temp=[]
        for j in range(p):
            temp+=[preds.iloc[i,j]]*weights[j]
            counts=cls.Counter(np.array(temp))
        label_temp=list(counts.most_common(1))
        label=label_temp[0][0]
        weighted_pred.append(label)
    return weighted_pred
            

In [41]:
final_pred=pred_ensembel(all_pred,[33,34,33])

In [28]:
all_pred=pd.read_csv('y_ensemble.csv')

In [42]:
np.mean(final_pred==y_test)

0    0.687597
dtype: float64

In [43]:
cls.Counter(final_pred)

Counter({3.0: 1, 7.0: 12137, 10.0: 2625})

However, we find that the accuracy is worse than the result of Xgboost. The reason is as following: The result of svm and random forest did not predict labels in country except USA and NDF. For the method of majority voting, the result is predominant by the svm and random forest. Thus, the proportion of correctly predict label of other countries will reduce.

Then, we get confusion matrix.

In [68]:
#confusion matrix
from sklearn.metrics import confusion_matrix

In [69]:
conf=pd.DataFrame(confusion_matrix(y_test,final_pred))

In [70]:
conf.index=range(12)

In [71]:
conf.columns=range(12)

In [72]:
conf

,0,1,2,3,4,5,6,7,8,9,10,11
0,0,0,0,0,0,0,0,19,0,0,7,0
1,0,0,0,0,0,0,0,45,0,0,50,0
2,0,0,0,0,1,0,0,25,0,0,19,0
3,0,0,0,0,1,0,0,90,0,0,40,1
4,0,0,0,1,3,0,0,188,0,0,93,0
5,0,0,0,0,1,0,0,95,0,0,50,0
6,0,0,0,0,0,0,0,113,0,0,63,0
7,0,0,0,0,0,0,0,8612,0,0,471,0
8,0,0,0,0,0,0,0,22,0,0,14,0
9,0,0,0,0,0,0,0,7,0,0,9,0


## Ensembling Probability

In [99]:
xg_prob=pd.read_csv('y_predprob_xgboost.csv')

In [100]:
xg_prob_array=np.array(xg_prob)

In [101]:
y_prob.shape

(14763L, 12L)

In [104]:
svm_prob=pd.read_csv('pred_prob_svm.csv',header=None)

In [105]:
svm_prob_array=np.array(svm_prob)

In [106]:
y_prob_array=np.array(y_prob)

In [108]:
all_prob=(svm_prob_array*33+xg_prob_array*34+y_prob_array*33)/100

In [110]:
prob_label=[np.argmax(x) for x in all_prob]

In [113]:
pd.DataFrame(all_pred).to_csv('probability.csv')